In [1]:
import os
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import f_oneway

current_dir = os.getcwd()
project_root = os.path.dirname(os.path.dirname(current_dir))
PROCESSED_DATA_PATH = os.path.join(project_root, 'data', 'processed')
RAW_DATA_PATH = os.path.join(project_root, 'data', 'raw')
OUTPUT_PATH = os.path.join(project_root, 'output')
REPORTS_TABLES_PATH = os.path.join(project_root, "reports", "tables")

In [2]:
df = pd.read_csv(PROCESSED_DATA_PATH + "/dummy.csv")

In [3]:
df

,COUNTRY,COMPANY CODE,INDUSTRY,ENVRTX21,ENVRTX37,ENVRTX02,ENVRTX22,ENVRTX39,ENVRTX23,ENVRTX18,...,OUTCMX11,OUTCMX12,OUTCMX13,OUTCMX14,OUTCMX15,OUTCMX16,OUTCMX17,JIT,Environmental,Category
0,BRA,1704,3,2.000000,4.0,2.000000,2.000000,3.0,4.000000,4.000000,...,3.0,2.000000,2.000000,3.000000,3.000000,3.000000,3.000000,Low,Low,Low JIT & Environmental
1,BRA,1713,1,3.000000,2.0,3.000000,1.000000,2.0,4.000000,2.000000,...,3.0,4.000000,1.000000,5.000000,4.000000,4.000000,4.000000,High,Low,Mainly JIT
2,BRA,1717,2,3.000000,4.0,5.000000,2.000000,4.0,5.000000,4.000000,...,3.0,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,High,High,High JIT & Environmental
3,BRA,1719,3,4.000000,2.0,4.000000,3.000000,3.0,4.000000,3.000000,...,3.0,4.000000,5.000000,5.000000,5.000000,4.000000,4.000000,High,Low,Mainly JIT
4,GER,401,2,4.666667,2.0,4.333333,4.333333,3.0,4.666667,3.333333,...,3.0,3.666667,3.666667,3.666667,3.666667,4.333333,4.333333,Low,Low,Low JIT & Environmental
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,USA,109,2,3.500000,2.5,3.500000,2.500000,3.5,4.000000,4.000000,...,3.0,4.000000,3.500000,4.500000,5.000000,4.000000,4.500000,High,High,High JIT & Environmental
171,USA,110,2,2.000000,1.5,3.000000,2.500000,2.5,3.500000,3.500000,...,2.5,4.000000,4.500000,4.000000,4.000000,4.500000,4.000000,Low,Low,Low JIT & Environmental
172,USA,111,1,2.000000,1.0,3.000000,2.000000,1.0,4.000000,2.000000,...,3.0,3.000000,3.000000,3.000000,4.000000,3.000000,4.000000,High,Low,Mainly JIT
173,SWI,1809,3,3.000000,2.0,3.000000,4.000000,3.0,5.000000,3.000000,...,3.0,4.000000,4.000000,4.000000,5.000000,5.000000,5.000000,High,Low,Mainly JIT


In [4]:
# Calculate the mean of performance (based on columns starting with "EPERFX")
eperfx_columns = [col for col in df.columns if col.startswith("EPERFX")]

# Check for non-numeric values
for col in eperfx_columns:
    if not pd.api.types.is_numeric_dtype(df[col]):
        print(f"Column {col} contains non-numeric data.")

# Check for NaN values
for col in eperfx_columns:
    if df[col].isna().any():
        print(f"Column {col} contains NaN values.")

df['Mean of Performance'] = df[eperfx_columns].mean(axis=1)

# Calculate the frequency of each category
category_counts = df['Category'].value_counts().reset_index()
category_counts.columns = ['Category', 'Frequency']

# Calculate the percentage of each category
total_rows = len(df)
category_counts['Percentage'] = (category_counts['Frequency'] / total_rows) * 100

# Aggregate data for each category
result_df = pd.merge(category_counts, df.groupby('Category')['Mean of Performance'].mean().reset_index(), on='Category')

# Rename columns for clarity
result_df.columns = ['Category', 'Frequency', 'Percentage', 'Mean of Performance']

# Sort by mean performance in descending order
result_df = result_df.sort_values(by='Mean of Performance', ascending=False)

# Reset the index
result_df = result_df.reset_index(drop=True)

# Print or work with 'result_df' as needed
print(result_df)


                   Category  Frequency  Percentage  Mean of Performance
0  High JIT & Environmental         68   38.857143             3.896242
1      Mainly Environmental         27   15.428571             3.767490
2                Mainly JIT         47   26.857143             3.502364
3   Low JIT & Environmental         33   18.857143             3.328844


In [5]:
# save results_df to csv
result_df.to_csv(PROCESSED_DATA_PATH + "/mean_perf.csv", index=False)

In [6]:
import pandas as pd

def dataframe_to_latex(df, file_name, label="tab:my_label", caption="My Caption"):
    """
    Convert a pandas DataFrame to a LaTeX table with specific column modifications,
    save it to a file, apply formatting to the numeric columns, and include label
    and caption for the table in LaTeX.

    Parameters:
    - df: pandas DataFrame to convert.
    - file_name: Name of the file to save the LaTeX code.
    - label: Label for the table in LaTeX.
    - caption: Caption for the table in LaTeX.
    """
    
    # Escape all LaTeX special characters in the DataFrame
    df = df.replace('&', '\\&', regex=True)
    
    # Format numbers in Python before converting to LaTeX
    df['Frequency'] = df['Frequency'].astype(int)
    df['Percentage'] = df['Percentage'].apply(lambda x: f"{x:.2f}")
    df['Mean of Performance'] = df['Mean of Performance'].apply(lambda x: f"{x:.2f}")

    # Create the LaTeX table code without the longtable format
    latex_code = df.to_latex(index=False, header=True, escape=False)

    # Wrap the tabular environment with the table float environment
    table_float = r'''\begin{table}[htbp]
    \centering
    \caption{''' + caption + r'''}
    \label{''' + label + r'''}
    ''' + latex_code + r'''
    \end{table}
    '''

    # Save to file
    with open(file_name, 'w') as file:
        file.write(table_float)


In [7]:
# # read csv
# df = pd.read_csv(PROCESSED_DATA_PATH + "/mean_perf.csv")

# latex_file_name = REPORTS_TABLES_PATH + "/performance_means.tex"
# dataframe_to_latex(df, latex_file_name, label="tab:your_label", caption="Practice Adoption and Environmental Performance")

In [18]:
# Step 1: Perform ANOVA to check for overall differences
anova_result = f_oneway(df[df['Category'] == 'High JIT & Environmental']['Mean of Performance'],
                        df[df['Category'] == 'Mainly Environmental']['Mean of Performance'],
                        df[df['Category'] == 'Mainly JIT']['Mean of Performance'],
                        df[df['Category'] == 'Low JIT & Environmental']['Mean of Performance'])

# Step 2: Check if ANOVA indicates significant differences
if anova_result.pvalue < 0.05:
    # Step 3: Perform Tukey's HSD test for pairwise comparisons
    tukey_results = pairwise_tukeyhsd(df['Mean of Performance'], df['Category'], alpha=0.05)

    # Display Tukey results
    print(tukey_results.summary())
else:
    print("No significant differences among categories based on ANOVA.")


                  Multiple Comparison of Means - Tukey HSD, FWER=0.05                  
         group1                   group2         meandiff p-adj   lower   upper  reject
---------------------------------------------------------------------------------------
High JIT & Environmental Low JIT & Environmental  -0.6308    0.0 -0.9028 -0.3589   True
High JIT & Environmental    Mainly Environmental   -0.134   0.59 -0.4101  0.1421  False
High JIT & Environmental              Mainly JIT  -0.3341 0.0092  -0.606 -0.0622   True
 Low JIT & Environmental    Mainly Environmental   0.4968 0.0002  0.1982  0.7955   True
 Low JIT & Environmental              Mainly JIT   0.2968 0.0478   0.002  0.5915   True
    Mainly Environmental              Mainly JIT  -0.2001 0.3072 -0.4987  0.0986  False
---------------------------------------------------------------------------------------


In [8]:
import pandas as pd
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

def tukey_to_latex(df, file_name, caption="Tukey Analysis", label="tab:your_label", alpha=0.05):
    """
    Perform ANOVA and Tukey HSD test on the given DataFrame, then convert the Tukey HSD test 
    results to a LaTeX table with a caption and label, and save it to a file.

    Parameters:
    - df: pandas DataFrame containing the data.
    - file_name: Name of the file to save the LaTeX code.
    - caption: Caption for the LaTeX table.
    - label: Label for the LaTeX table.
    - alpha: Significance level for Tukey HSD test.
    """
    # Perform ANOVA
    anova_result = f_oneway(
        df[df['Category'] == 'High JIT & Environmental']['Mean of Performance'],
        df[df['Category'] == 'Mainly Environmental']['Mean of Performance'],
        df[df['Category'] == 'Mainly JIT']['Mean of Performance'],
        df[df['Category'] == 'Low JIT & Environmental']['Mean of Performance']
    )
    
    # Initialize the LaTeX code for the table with caption and label
    latex_code = f"\\begin{{table}}[htbp]\n\\centering\n\\caption{{{caption}}}\n\\label{{{label}}}\n\\begin{{tabular}}{{llrrrrr}}\n\\toprule\n"

    # Check if ANOVA is significant
    if anova_result.pvalue < alpha:
        # Perform Tukey HSD
        tukey_results = pairwise_tukeyhsd(df['Mean of Performance'], df['Category'], alpha=alpha)
        # Convert Tukey HSD results to a DataFrame
        tukey_df = pd.DataFrame(data=tukey_results._results_table.data[1:], columns=tukey_results._results_table.data[0])
        
        # Escape ampersands in group names and round all numeric columns to two decimal places
        tukey_df['group1'] = tukey_df['group1'].str.replace('&', '\\&', regex=False)
        tukey_df['group2'] = tukey_df['group2'].str.replace('&', '\\&', regex=False)
        numeric_cols = tukey_df.select_dtypes(include=['float64', 'float32']).columns
        tukey_df[numeric_cols] = tukey_df[numeric_cols].round(2)

        # Assuming 'tukey_df' is your DataFrame and you want to drop columns named 'col_to_drop1', 'col_to_drop2', etc.
        columns_to_drop = ['meandiff']
        tukey_df = tukey_df.drop(columns_to_drop, axis=1)
        
        # Convert DataFrame to LaTeX code, remove the tabular environment from the generated LaTeX table and append the content
        tukey_latex = tukey_df.to_latex(index=False, header=True, escape=False, formatters={
            # 'meandiff': '{:0.2f}'.format,
            'p-adj': '{:0.2f}'.format,
            'lower': '{:0.2f}'.format,
            'upper': '{:0.2f}'.format,
        })
        
        # Remove the tabular environment from the generated LaTeX table and append the content
        tukey_latex = tukey_latex.split("\n", 2)[-1].rsplit("\n", 1)[0]  # Remove the first and last lines containing tabular environment
        latex_code += tukey_latex
    else:
        latex_code += "No significant differences found by ANOVA.\n"

    # Correctly finish the LaTeX table setup
    latex_code += "\\end{table}"

    # Save to file
    with open(file_name, 'w') as file:
        file.write(latex_code)

# Example usage:
# Ensure that df is defined in your workspace before calling this function
# tukey_to_latex(df, 'tukey_results.tex')


In [9]:
tukey_to_latex(df, REPORTS_TABLES_PATH + '/tukey_results.tex', caption="Multiple Comparison of Means - Tukey HSD, FWER=0.05", label="tab:your_label", alpha=0.05)

In [10]:
# import statsmodels.api as sm

# # Fit a linear regression model with "ACCTGX51" as a predictor
# X = df[['ACCTGX51']]
# X = sm.add_constant(X)  # Add a constant term (intercept)
# y = df['Mean of Performance']

# model = sm.OLS(y, X).fit()

# # Print regression summary
# print(model.summary())


In [11]:
# import statsmodels.api as sm

# # Fit an ANCOVA model
# model = sm.OLS.from_formula('"Mean of Performance" ~ C(Category) + ACCTGX51', data=df).fit()

# # Perform ANCOVA
# anova_results = sm.stats.anova_lm(model, typ=2)

# # Check for significance
# if anova_results['PR(>F)']['C(Category)'] < 0.05:
#     # If the categorical variable is significant, you can proceed with post hoc tests.
#     # Otherwise, you may conclude there are no significant differences.
#     tukey_results = pairwise_tukeyhsd(df['Mean of Performance'], df['Category'], alpha=0.05)
#     print(tukey_results.summary())
# else:
#     print("No significant differences among categories based on ANCOVA.")


In [12]:
# import statsmodels.api as sm
# from statsmodels.formula.api import ols

# # Fit ANCOVA model with ACCTGX51 as a covariate
# formula = 'Performance_Mean ~ Category + ACCTGX51'
# model = ols(formula, data=df).fit()

# # Perform ANOVA
# anova_table = sm.stats.anova_lm(model, typ=2)
# print(anova_table)

# # Perform pairwise comparisons with Bonferroni correction
# pairwise_comparisons = model.t_test_pairwise('Category').result_frame
# pairwise_comparisons